In [2]:
# imports
import pandas as pd
import numpy as np

# GDP data reading and cleaning

In [3]:
gdp_path = '../../data/gdpexpenditurebased/36100104.csv'
gdp = pd.read_csv(gdp_path)

C:\Users\sidhu\AppData\Local\Temp/ipykernel_20424/578221398.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  gdp = pd.read_csv(gdp_path)


In [44]:
gdp.head(2)

,REF_DATE,GEO,DGUID,Prices,Seasonal adjustment,Estimates,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,Jan-61,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Final consumption expenditure,Dollars,81,millions,6,v62305723,1.1.1.1,273818.0,NaN,NaN,NaN,0
1,Jan-61,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Household final consumption expenditure,Dollars,81,millions,6,v62305724,1.1.1.2,182300.0,NaN,NaN,NaN,0


### Extracted only required columns

In [45]:
gdp_subset = gdp[['REF_DATE', 'GEO', 'Prices', 'Estimates', 'UOM', 'VALUE']]
gdp_subset.head()

,REF_DATE,GEO,Prices,Estimates,UOM,VALUE
0,Jan-61,Canada,Chained (2012) dollars,Final consumption expenditure,Dollars,273818.0
1,Jan-61,Canada,Chained (2012) dollars,Household final consumption expenditure,Dollars,182300.0
2,Jan-61,Canada,Chained (2012) dollars,Goods,Dollars,81917.0
3,Jan-61,Canada,Chained (2012) dollars,Durable goods,Dollars,7346.0
4,Jan-61,Canada,Chained (2012) dollars,Semi-durable goods,Dollars,10174.0


In [46]:
gdp_subset['Prices'].unique()

array(['Chained (2012) dollars', 'Current prices',
       'Contributions to percent change',
       'Contributions to percent change, annualized',
       'Chained (2007) dollars'], dtype=object)

In [58]:
from datetime import datetime
datetime.strptime('Feb-2010', '%b-%Y')

datetime.datetime(2010, 2, 1, 0, 0)